In [56]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [57]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [58]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)


{'result': [{'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'content': '東京駅(東京都)周辺のイベント｜ウォーカープラス 千代田区 中央区 ウォーカープラス 東京駅(東京都)周辺のイベント 東京駅(東京都)周辺のイベント 今日 明日 今週末 千代田区 中央区 アニメ「鬼滅の刃」柱展 ーそして無限城へー終了間近 2024年11月2日(土)～2025年3月2日(日) 江戸にひかり大丸有開催中 2025年1月9日(木)～3月7日(金) 二重橋前駅(東京都), 東京駅(東京都), 大手町駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都) 生誕120年 宮脇綾子の芸術 見た、切った、貼った開催中 2025年1月25日(土)～3月16日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) 1日限りの光の祭典「アイスキャンドルフェスティバル」と山中湖ダイヤモンド富士と河口湖湖畔 大石公園終了間近 2025年2月23日(日) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 丸の内エリアプロジェクションマッピング 東京大浮世絵終了間近 2025年2月21日(金)～25日(火) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 大手町駅(東京都), 東京駅(東京都), 二重橋前駅(東京都), 神田駅(東京都), 三越前駅(東京都) Otemachi One Illumination終了間近 2024年11/8(金)～2025年2月下旬 大手町駅(東京都), 竹橋駅(東京都), 神田駅(東京都), 二重橋前駅(東京都), 東京駅(東京都) 京橋エドグラン クリスマスイルミネーション2024終了間近 2024

In [59]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [60]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [61]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [62]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [63]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)


東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- **東京都**の面積は約2,194平方キロメートルです。
- **沖縄県**の面積は約2,283平方キロメートルです。

したがって、沖縄県の方が東京都よりも広いという結果になります。


In [64]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内に東京駅周辺で開催されるイベントについて、以下の情報があります：

1. **【東京駅近商業施設】秋のイベント**
   - 日程: 2023年9月14日(木)・15日(金)
   - 内容: 「観光PR・物産展 in 舩の内トラストシティ」で開催されます。
   - 詳細: [こちらをチェック](https://tabizine.jp/article/547199/)

2. **東京の秋祭りとイベント**
   - 内容: 2023年9月に東京で行われる秋の祭りやイベントを紹介しています。
   - 詳細: [もっと見る](https://www.enjoytokyo.jp/feature/matsuri_autumn/cal_sept_tokyo/)

3. **9月のイベントカレンダー**
   - 内容: 秋の主なイベントや、2023年の東京での催し物を特集した情報です。
   - 詳細: [カレンダーはこちら](http://www.winriver.net/tokyo/event/event2023/event202309.php)

4. **【関東・東京】秋の祭りや花火**
   - 日程: 2023年9月中で多くの祭りや花火イベントが行われます。
   - 詳細: [詳細情報](https://tabiiro.jp/higaeri/article/september2023-event/)

これらのイベントは、東京駅周辺で楽しめるフレッシュなコンテンツですので、ぜひチェックしてみてください。


In [67]:
# チャットボットへの組み込み
tools = define_tools()

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちわ！'

こんにちわ！どういったことをお手伝いしましょうか？


'質問:東北6県は？'

東北地方は、日本の北部に位置し、以下の6つの県から構成されています：

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

これらの県は、自然の美しさや豊かな文化、歴史的な背景を持つ地域です。


'質問:福島県のお土産について検索した結果を教えて'

福島県のお土産についての情報をいくつかご紹介します。

1. **福島のお土産選びのガイド**:
   - 福島県の名産品や特産品が紹介されています。特に「お塩」や「お菓子」、「地酒」などが人気です。福島の地域の食材や風土に根ざしたお土産が多く、手に入れる価値があります。
   - 参考URL: [福島の名産品 - Sakidori](https://sakidori.co/article/1320730)

2. **地方固有の食品やお菓子**:
   - 地元産の大豆や米を使用した食品、お菓子が数多くあり、人気の選択肢としては「ラーメン」や「お茶」、特に「野菜の加工品」が挙げられます。また「お酒」も特に推薦されるお土産です。
   - 参考URL: [福島県の愉しみ - amaびじん](https://amabijin.com/article/14865)

3. **地元の人が推すお土産ランキング**:
   - 地元の人々が選ぶ福島のお土産トップ10が掲載されており、地域で愛されている特産品が集められています。
   - 参考URL: [旅行ガイド](https://www.jalan.net/news/article/93586/)

4. **旅行者向けのおすすめお土産**:
   - 最新のトレンドを反映した福島のお土産が紹介されています。各地での定番や新しい商品を押さえることができる情報があります。
   - 参考URL: [Trip.com お土産ガイド](https://jp.trip.com/guide/shopping/福島+お土産.html)

5. **イベントや販売所情報**:
   - 現地で購入可能な特産品についての詳細や、販売店の情報がまとめられています。県内の市場や観光地で見つけられる商品についても触れられています。
   - 参考URL: [macaroni お土産特集](https://macaro-ni.jp/159948)

これらの情報をもとに、福島県ならではのお土産を楽しく選んでみてください。どれも地元の魅力が詰まった品々です。

---ご利用ありがとうございました！---
